In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

import pandas as pd

In [3]:
wine = load_wine()
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

sklearn에서 제공하는 wine 에는 다음과 같은 필드가 있다.

* DESCR: 데이터 집합의 상세정보
* data: 와인 성분 데이터 (설명변수)
* feature_names: 와인 성분명
* target: 와인의 품종 데이터 (목적변수)
* target_names: 와인의 품종이름

In [4]:
pd.DataFrame(wine.data, columns=wine.feature_names)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


In [5]:
wine.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

다음은 목적 변수다. 0 부터 2까지의 값을 갖는 numpy배열이다.

일단 0과 1 분류만 해보도록 하자. 그리고, 각각의 데이터를 훈련데이터, 테스트 데이터로 나눈다.

In [6]:
wine_data = wine.data[0:130]
wine_target = wine.target[0:130]

train_X, test_X, train_Y, test_Y = train_test_split(wine_data, wine_target, test_size=0.2)

그리고 각각의 데이터를, 파이토치가 다룰 수 있는 형태로 정리한다.

In [7]:
# 데이터를 텐서 형태로 변환
train_X = torch.from_numpy(train_X).float()
train_Y = torch.from_numpy(train_Y).long()

test_X = torch.from_numpy(test_X).float()
test_Y = torch.from_numpy(test_Y).float()

그리고 설명변수와 목적변수의 텐서를 합친다. 이를 미니배치로 분할한다. 미니배치 사이즈는 16이다.

In [8]:
train = TensorDataset(train_X, train_Y)
train_loader = DataLoader(train, batch_size=16, shuffle=True)

In [9]:
train[0]

(tensor([1.3560e+01, 1.7300e+00, 2.4600e+00, 2.0500e+01, 1.1600e+02, 2.9600e+00,
         2.7800e+00, 2.0000e-01, 2.4500e+00, 6.2500e+00, 9.8000e-01, 3.0300e+00,
         1.1200e+03]), tensor(0))

train 첫번째 데이터를 살짝 엿보면, 각각의 설명변수와 목적변수가 따로 담겨 있는 것을 알 수 있다.

신경망 구성

만들어볼 신경망 구성은 다음과 같다. 입력층, 중간층, 출력층이 각각 하나씩 있는 신경망을 구성한다. 입력층의 노드 개수는 13개 (설명변수가 13개 니깐)고, 중간층 노드의 개수는 96개 (내맘) 출력층 노드의 수는 2개 (목적변수가 0, 1 이니깐…) 다. 마지막 출력층에서는 0일 확률과 1일 확률을 뱉어낼 것이다.

In [10]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 입력층과 중간층 사의의 정의. 13개의 입력변수를 받아서 96개의 중간층 노드를 만든다.
        self.fc1 = nn.Linear(13, 96)
        # 중간층과 출력층 사의의 정의. 96개의 중간변수를 받아서 2개의 최종 노드를 만든다.
        self.fc2 = nn.Linear(96, 2)

    # 순전파
    def forward(self, x):
        # relu함수로 입력층을 변환한다.
        x = F.relu(self.fc1(x))
        # 이를 중간층으로 넘긴다.
        x = self.fc2(x)
        # 출력 함수로 log_softmax를 사용한다.
        return F.log_softmax(x, dim=1)

# 선언
model = Net()

torch.nn.Module: 모든 신경망 모듈의 기본이 되는 클래스다. 이 안에 각 층, 함수, 신경망구조를 정의한다.

torch.nn.Linear: 입력데이터에 대해서 선형 변환
y=Ax+b
를 변환한다.

torch.nn.functional.relu: ReLU

torch.nn.fucntional.log_softmax: log softmax를 구현했다.

모형학습

In [11]:
# 오차함수. 분류 문제에는 교차 함수로 크로스 엔트로피를 사용한다고 했었다.
criterion = nn.CrossEntropyLoss()

# 최적화 담당. 경사 하강법을 적용하였다. 학습률은 0.01.
optimizer = optim.SGD(model.parameters(), lr=0.01)

#학습 300회 ㄱㄱ 씽
for epoch in range(300):
  # 누적 오차를 담당할 변수
  total_loss = 0
  
  # 아까 만들어놓은 미니 배치에서 각각 변수를 꺼내온다.
  for train_x, train_y in train_loader:
    # 각각의 값을 변수로 만든다.
    train_x, train_y = Variable(train_x), Variable(train_y)

    # 경사 초기화
    optimizer.zero_grad()

    # 순전파
    output = model(train_x)

    # 오차계산. ouutput 과 train_y 비교
    loss = criterion(output, train_y)

    # 역전파 계산
    loss.backward()

    # 가중치 업데이트. learning rate만큼 
    optimizer.step()

    #총 오차 업데이트
    total_loss += loss.data.item()

  # 10 회 마다 현재 오차를 출력
  if (epoch + 1) % 10 == 0:
    print(epoch + 1, total_loss)

10 4.863840103149414
20 4.8596853613853455
30 4.853436529636383
40 4.850582301616669
50 4.847133815288544
60 4.85513961315155
70 4.851650714874268
80 4.842878460884094
90 4.845352351665497
100 4.844212532043457
110 4.848921597003937
120 4.852788209915161
130 4.853225946426392
140 4.853307127952576
150 4.835079491138458
160 4.854700863361359
170 4.8483089208602905
180 4.838370442390442
190 4.864116430282593
200 4.843770384788513
210 4.839619874954224
220 4.84391587972641
230 4.8485846519470215
240 4.865064918994904
250 4.852663218975067
260 4.844613969326019
270 4.852788925170898
280 4.852721154689789
290 4.84778618812561
300 4.843870222568512


정확도를 계산해보자

In [12]:
# 테스트 데이터를 집어 넣는다.
test_x, test_y = Variable(test_X), Variable(test_Y)
# 테스트 데이터를 집어 넣어서 학습 시킨 다음, max 값(확률이 더 높은 값)을 출력한다.
result = torch.max(model(test_x).data, 1)[1]
# 정확도 계산
accuracy = sum(test_y.data.numpy() == result.numpy()) / len(test_y.data.numpy())
print("accuracy= ",accuracy)

accuracy=  0.6538461538461539
